In [ ]:
# 1. install docker
#    go to https://docs.docker.com/
#    username - willtun  pw: StGeorge231008!


# https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

In [3]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
from collections import namedtuple
from itertools import count
from PIL import Image

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


In [ ]:
env = gym.make('CartPole-v0').unwrapped

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)
        self.head = nn.Linear(448, 2)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

In [ ]:
#resize = T.Compose([T.ToPILImage(),
#                    T.Resize(40, interpolation=Image.CUBIC),
#                    T.ToTensor()])

# This is based on the code from gym.
screen_width = 600


def get_cart_location():
    world_width = env.x_threshold * 2
    scale = screen_width / world_width
    return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART


def get_screen():
    screen = env.render(mode='rgb_array').transpose(
        (2, 0, 1))  # transpose into torch order (CHW)
    # Strip off the top and bottom of the screen
    screen = screen[:, 160:320]
    view_width = 320
    cart_location = get_cart_location()
    if cart_location < view_width // 2:
        slice_range = slice(view_width)
    elif cart_location > (screen_width - view_width // 2):
        slice_range = slice(-view_width, None)
    else:
        slice_range = slice(cart_location - view_width // 2,
                            cart_location + view_width // 2)
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, :, slice_range]
    # Convert to float, rescare, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return screen.unsqueeze(0).to(device)


env.reset()
plt.figure()
#plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
#           interpolation='none')
#plt.title('Example extracted screen')
plt.show()

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

policy_net = DQN().to(device)
target_net = DQN().to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)


steps_done = 0


# RL version 1.

In [ ]:
env = gym.make('CartPole-v0')

In [ ]:
env.reset()

In [ ]:
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample())

In [ ]:
from gym import envs

In [ ]:
print(envs.registry.all())

# Training a robot to walk

In [ ]:
walking_env = gym.make("BipedalWalker-v2")

In [ ]:
for episode in range(100):
    observation = env.reset()
    for t in range(10000):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("{} time steps taken for episode".format(t+1))
            break

# Videogame bot that plays a car rracing game

In [4]:
import gym
import random

In [ ]:
# https://medium.freecodecamp.org/how-to-build-an-ai-game-bot-using-openai-gym-and-universe-f2eb9bfbb40a
# pip uninstall gym
# pip install gym==0.9.5
#https://github.com/openai/universe/issues/139

In [5]:
import universe


In [6]:
env3 = gym.make('flashgames.NeonRace-v0')

[2018-10-17 16:57:42,732] Making new env: flashgames.NeonRace-v0


In [7]:
env3.configure(remotes=1)

[2018-10-17 16:57:45,659] Writing logs to file: /tmp/universe-57870.log
[2018-10-17 16:57:45,777] Ports used: dict_keys([])
[2018-10-17 16:57:45,779] [0] Creating container: image=quay.io/openai/universe.flashgames:0.20.28. Run the same thing by hand as: docker run -p 5900:5900 -p 15900:15900 --privileged --cap-add SYS_ADMIN --ipc host quay.io/openai/universe.flashgames:0.20.28
[2018-10-17 16:57:45,789] Image quay.io/openai/universe.flashgames:0.20.28 not present locally; pulling


APIError: 500 Server Error: Internal Server Error ("b'{"message":"Get https://quay.io/v2/: net/http: request canceled while waiting for connection (Client.Timeout exceeded while awaiting headers)"}'")

In [ ]:
observation_n = env.reset()

In [ ]:
left = [('KeyEvent', 'ArrowUp', True),
        ('KeyEvent', 'ArrowLeft', True), 
        ('KeyEvent', 'ArrowRight', False)]

right = [('KeyEvent', 'ArrowUp', True),
        ('KeyEvent', 'ArrowLeft', False), 
        ('KeyEvent', 'ArrowRight', True)]


forward = [('KeyEvent', 'ArrowUp', True),
        ('KeyEvent', 'ArrowRight', False), 
        ('KeyEvent', 'ArrowLeft', False),
        ('KeyEvent', 'n', True)]

In [ ]:
turn = 0 
rewards = [] 
buffer_size = 100
action = forward

In [ ]:
while True:
    turn -= 1
    if turn <= 0:
        action = forward
        turn = 0
    action_n = [action for ob in observation_n]
    observation_n, reward_n, done_n, info = env3.step(action=action_n)
    rewards += [reward_n[0]]
    if len(rewards) >= buffer_size:
        mean = sum(rewards)/len(rewards)
        
        if mean == 0:
            turn = 20
            if random.random() < 0.5:
                action =right
            else:
                action=left
        rewards = []

env.render()

In [ ]:
env3.configure(remotes=1)